In [ ]:
using StatsBase, Plotly, BenchmarkTools

In [ ]:
struct Bairro
    nResidencias::Integer # numero de residencias naquele bairro
    nPessoas::Integer # numero de pessoas naquele bairro
    residencias::Array{T} where T <: Integer # indice das residencias do bairro
    pessoas::Array{T} where T <: Integer # indice das pessoas do bairro
    distancias::Array{T} where T <: Number # distancia media entre uma pessoa do bairro e os demais bairros
end

struct Residencia
    n::Integer # numero de pessoas naquela residencia
    pessoas::Array{T} where T <: Integer # indice das pessoas da residencia
    bairro::Integer # indice do bairro que a residencia pertence
    posicao::Array{T} where T <: Number # posicao da residencia no mapa
end

struct Social
    n::Integer # numero de pessoas naquela rede social
    pessoas::Array{T} where T <: Integer # pessoas que coimpoe a rede social
end

struct Pessoas
    residencias::Array{T} where T <: Integer # lista com indices da residencia onde cada uma das pessoas residem
    bairros::Array{T} where T <: Integer # lista com indices do bairro onde cada uma das pessoas residem
    social::Array{T} where T <: Integer # lista com indices da rede social na qual cada uma das pessoas pertencem
    posicao::Array{T, 2} where T <: Number # posicao de cada pessoa
end

struct Populacao
    rodada::Integer # n da rodada (nome da pasta com arquivos)
    n::Integer # tamanho da população
    I0::Integer # quantidade inicial de infectados
    estadoInicial::Array{T} where T <: Integer # estado inicial para cada individuo
    estadoAtual::Array{T} where T <: Integer # estado inicial de cada individuo
    pessoas::Pessoas # pessoas que compoem a populacao
    residencias::Array{Residencia} # lista com as residencias
    bairros::Array{Bairro} # lista de bairros
    sociais::Array{Social} # lista de redes sociais
end

In [ ]:
function selectiveRand(v)
    """
        Função para geração de números aleatórios de acordo com um vetor booleano
    
        Parametros:
            v: vetor booleano
        
        Saida:
            aux: vetor com um número aleatório nas entradas verdadeiras de v o 0 nas demais
    """
    aux = zeros(length(v))
    aux[v] .= rand(sum(v))
    return aux
end

function rowWiseNorm(A)
    """
        Função para calcular a norma dos vetores linha de uma matriz
    """
    return sqrt.(sum(abs2, A, dims=2)[:, 1])
end

In [ ]:
function calculaDistanciaFina(populacao::Populacao, bairro::Bairro, suscetiveisBairro, infectadosBairro, fKernel; T=Float16)
    """
        Cálculo da distancia entre as pessoas de um mesmo bairro
    """
    aux = zeros(T, length(suscetiveisBairro), length(infectadosBairro))
    Threads.@threads for i in 1:length(suscetiveisBairro)
        aux[i, :] += fKernel(populacao.pessoas.posicao[infectadosBairro] .- populacao.pessoas.posicao[suscetiveisBairro[i], :]')
    end
    return aux
end

function calculaDistancia(populacao::Populacao, suscetiveis, infectados, fKernel)
    """
        Calculo da distância entre todas as pessoas, tomando a distnacia media entre pessoas de bairros diferentes
    """
    aux = zeros(populacao.n)
    infectadosBairros = [sum(infectados[i.pessoas]) for i in populacao.bairros]
    for bairro in populacao.bairros
        suscetiveisBairro = bairro.pessoas[suscetiveis[bairro.pessoas]]
        infectadosBairro = bairro.pessoas[infectados[bairro.pessoas]]
        aux[suscetiveisBairro] .+= ones(length(suscetiveisBairro)) .* sum(infectadosBairros .* bairro.distancias)
        aux[suscetiveisBairro] .+= sum(calculaDistanciaFina(populacao, bairro, suscetiveisBairro, infectadosBairro, fKernel), dims=2)[:, 1]
    end
    return aux[suscetiveis]
end

function escreveDistancias(populacao, rodada, fKernel; T=Float16)
    """
        Calcula e escreve a matriz de distancias entre pessoas do mesmo bairro, para todos os bairros
    """
    rm(joinpath("saidas", string(rodada)), recursive=true, force=true)
    mkdir(joinpath("saidas", string(rodada)))
    dir = joinpath("saidas", string(rodada), "dist")
    mkdir(dir)
    for i in 1:length(populacao.bairros)
        bairro = populacao.bairros[i]
        file = open(joinpath(dir, string(i) * ".eps"), "w")
        aux = Array{T, 2}(calculaDistanciaFina(populacao, bairro, bairro.pessoas, bairro.pessoas, fKernel))
        write(file, aux)
        close(file)
        aux = nothing
        file = nothing
        GC.gc()
    end
end

In [ ]:
function miniPassoMatricial(estados::Array{T} where T <: Integer)
    """
        Passo matricial para calcular exposicao de uma rede completa
    """
    popSuscetiveis = estados.==1
    popInfectados = estados.==2
    
    n = length(estados)
    nInfectados = sum(popInfectados)
    nSuscetiveis = sum(popSuscetiveis)

    expostos = zeros(Bool, n, n)
    expostos[popSuscetiveis, popInfectados] .= true
    return sum(expostos, dims=2)[:, 1]
end

function abreEps(rodada, i, T=Float16)
    """
        Abre a matriz de distancias de um bairro
    """
    file = open(joinpath("saidas", string(rodada), "dist", string(i) * ".eps"))
    aux = reinterpret(T, read(file))
    close(file)
    n = Int(sqrt(length(aux)))
    return reshape(aux, (n, n))
end

function leDistancia(populacao::Populacao, suscetiveis, infectados, fKernel)
    """
        Calcula soma das distancias entre os infectados e os suscetiveis, 
        tomando a distancia media entre pessoas de bairros diferentes e lendo a matriz de distancias
        para pessoas do mesmo bairro.
    """
    aux = zeros(populacao.n)
    infectadosBairros = [sum(infectados[i.pessoas]) for i in populacao.bairros]
    Threads.@threads for i in 1:length(populacao.bairros)
        bairro = populacao.bairros[i]
        suscetiveisBairro = suscetiveis[bairro.pessoas]
        infectadosBairro = infectados[bairro.pessoas]
        aux[bairro.pessoas[suscetiveisBairro]] .+= ones(sum(suscetiveisBairro)) .* sum(infectadosBairros .* bairro.distancias)
        aux[bairro.pessoas[suscetiveisBairro]] .+= sum(abreEps(populacao.rodada, i)[suscetiveisBairro, infectadosBairro], dims=2)[:, 1]
    end
    return aux[suscetiveis]
end

function passoMisto(populacao::Populacao, 
        α::Array{T} where T <: Number, β::Array{T} where T <: Number, θ::Array{T} where T <: Number, 
        γ::Number, δ::Number, fKernel)
    """
        Entrada:
            populacao: 
            α: taxa de transmissão residencial
            β: taxa de transmissão social
            θ: taxa de transmissão global
            γ: probabilidade de não recuperação
            δ: tamanho do passo temporal
            fKernel: ?
    """
    popSuscetiveis = populacao.estadoAtual.==1
    popInfectados = populacao.estadoAtual.==2
    
    contatos = zeros(populacao.n)
    Threads.@threads for i in populacao.residencias
        contatos[i.pessoas] .+= miniPassoMatricial(populacao.estadoAtual[i.pessoas]) .* α[i.pessoas]
    end
    
    Threads.@threads for i in populacao.sociais
        contatos[i.pessoas] .+= miniPassoMatricial(populacao.estadoAtual[i.pessoas]) .* β[i.pessoas]
    end
    
    contatos[popSuscetiveis] .+= leDistancia(populacao, popSuscetiveis, popInfectados, fKernel) .* θ[popSuscetiveis]
    
    prob = exp.(- δ .* contatos)
    
    novosInfectados = selectiveRand(popSuscetiveis) .> prob
    novosRecuperados = selectiveRand(popInfectados) .> γ
    
    infectados = ((popInfectados .& (.~novosRecuperados)) .| novosInfectados)
    suscetiveis = (popSuscetiveis .& (.~novosInfectados))

    return 3 .* ones(Int, populacao.n) .- 2 .* suscetiveis .- infectados
end

function evolucaoMista(
        populacao::Populacao, tempos::AbstractArray{T} where T <: Number, nSim::Integer, 
        α::Array{T} where T <: Number, β::Array{T} where T <: Number, θ::Array{T} where T <: Number, γ::Number, fKernel)
    """
        Entrada:
            resumoPop: 
            tempos:
            α: taxa de transmissão residencial
            β: taxa de transmissão social
            θ: taxa de transmissão global
            γ: parâmetro da exponencial de não recuperação
            δ: tamanho do passo temporal
            fKernel: ?
    """    
    nT = length(tempos)
    passos = tempos[2:end] - tempos[1:(end-1)]
    Γ = exp.(- γ .* passos)
    
    S = zeros(nSim, nT)
    I = zeros(nSim, nT)
    R = zeros(nSim, nT)
    
    S[:, 1] .= populacao.n - populacao.I0
    I[:, 1] .= populacao.I0
    
    for j in 1:nSim
        populacao.estadoAtual .= populacao.estadoInicial
        for (k, δ) in enumerate(passos)
            @time populacao.estadoAtual .= passoMisto(populacao, α, β, θ, Γ[k], δ, fKernel)
            GC.gc()
            
            S[j, k+1] = sum(populacao.estadoAtual .== 1)
            I[j, k+1] = sum(populacao.estadoAtual .== 2)
            R[j, k+1] = sum(populacao.estadoAtual .== 3)
        end
    end
    return S,I,R
end

In [ ]:
function geraPopulacaoAleatoria(rodada, n, I0, nResidencias, nSociais, bairrosShape, fKernel)
    """
        Gera populacao aleatoria de maneira simplificada
    """
    nBairros = prod(bairrosShape)
    pop0 = ones(Int, n)
    
    # escolhe pessoas aleatorias para comecarem infectadas
    pop0[StatsBase.sample(1:n, I0, replace=false)] .= 2;

    # distribui as pessoas nas residencias
    residencia = rand(1:nResidencias, n)
    
    # determina que pessoas estão em cada residencia
    residencias = [Int[] for i in 1:nResidencias]   
    Threads.@threads for i in 1:nResidencias
        append!(residencias[i], (1:n)[residencia .== i])
    end
    
    # atribui uma posicao e determina o bairro que a residencia pertence
    residenciasPos = rand(nResidencias, 2) .* bairrosShape'
    residenciasBai = [
        ceil(Int, residenciasPos[i, 1]) + 
        floor(Int, residenciasPos[i, 2]) * bairrosShape[1]
        for i in 1:nResidencias
    ]
    
    # cria listas com residencias
    residencias = [Residencia(length(j), j, residenciasBai[i], residenciasPos[i, :]) for (i, j) in enumerate(residencias)]
    
    # cria redes sociais
    social = rand(1:nSociais, n)
    sociais = [Social(sum(social .== i), (1:n)[social .== i]) for i in 1:nSociais]
    
    # determina a posicao e bairro de cada pessoa
    posicao = zeros(n, 2)
    bairro = zeros(Int, n)
    Threads.@threads for i in 1:n
        posicao[i, :] .= residenciasPos[residencia[i], :]
        bairro[i] = residenciasBai[residencia[i]]
    end
    
    pessoas = Pessoas(residencia, bairro, social, posicao)
    
    # identifica pessoas e residencias de cada bairro
    bairrosResidencias = [(1:nResidencias)[residenciasBai .== i] for i in 1:nBairros]
    bairrosPessoas = [(1:n)[bairro .== i] for i in 1:nBairros]
    
    # calcula o centro do bairro e a distancia media entre os bairros
    centros = vcat([mean(posicao[i, :], dims=1) for i in bairrosPessoas]...)
    dist = hcat([fKernel(centros .- centros[i, :]') for i in 1:nBairros]...)
    replace!(dist, NaN=>0.)
    
    #cria lista de bairros e populacao
    bairros = [
        Bairro(
            length(bairrosResidencias[i]),
            length(bairrosPessoas[i]),
            bairrosResidencias[i],
            bairrosPessoas[i],
            dist[i, :]
            ) for i in 1:nBairros
    ]
    populacao = Populacao(rodada, n, I0, pop0, copy(pop0), pessoas, residencias, bairros, sociais)
    
    # escreve matriz de distancias das pessoas de cada bairro
    escreveDistancias(populacao, rodada, fKernel)
    return populacao
end

In [ ]:
function powerDecay(a::Array)
    b = rowWiseNorm(a)
    return b ./ (b .+ 1.5);
end

In [ ]:
function foo(k, bairros)
    rodada = length(readdir("saidas")) + 1
    k = floor(Int, k / 100)

    n = 100 * k
    I0 = 1 * k
    nResidencias = 30 * k
    nSociais = 1 * k
    return geraPopulacaoAleatoria(rodada, n, I0, nResidencias, nSociais, bairros, powerDecay);
end

In [ ]:
bairrosShape = [5, 2]
@time populacao = foo(1e+4, bairrosShape);

In [ ]:
data = Dict([
    "x" => populacao.pessoas.posicao[:, 1],
    "autobinx" => false,
    "xbins" => Dict([
        "start" => 0,
        "end" => bairrosShape[1],
        "size" => 1
    ]),
    "y"=>populacao.pessoas.posicao[:, 2],
    "autobiny" => false,
    "ybins" => Dict([
        "start" => 0,
        "end" => bairrosShape[2],
        "size" => 1
    ]),
]);

Plotly.plot(Plotly.histogram2d(data))

In [ ]:
data = Dict([
    "x"=>populacao.pessoas.posicao[:, 1],
    "y"=>populacao.pessoas.posicao[:, 2]
]);

Plotly.plot(Plotly.histogram2dcontour(data))

In [ ]:
# tem que rever esses parâmetros
α = 1e-1 * ones(populacao.n)
β = 1e-4 * ones(populacao.n)
@time θ = 1e-1 ./ leDistancia(populacao, ones(Bool, populacao.n), ones(Bool, populacao.n), powerDecay);
GC.gc()
γ = 0.2;

In [ ]:
@time a = evolucaoMista(populacao, Array(1:10), 1, α, β, θ, γ, powerDecay)